In [4]:
#importing required libraries
import boto3
from pandas import DataFrame,read_csv


In [5]:
data=read_csv(r"C:\Users\user\Downloads\data_sample.csv")

In [6]:
print(data)

        id        date   time continent_code   country_name country_code  \
0       34  03-02-2007  Night            NaN  United States           US   
1       42     3/22/07    NaN            NaN  United States           US   
2       56  04-06-2007    NaN            NaN  United States           US   
3       59     4/14/07    NaN            NaN         Canada           CA   
4       61     4/15/07    NaN            NaN  United States           US   
...    ...         ...    ...            ...            ...          ...   
1688  7535  12-07-2015    NaN            NaN  United States           US   
1689  7537     2/22/16  00:00            NaN  United States           US   
1690  7539     2/23/16    NaN            NaN  United States           US   
1691  7540     2/26/16  21:06            NaN  United States           US   
1692  7541  03-02-2016  08:00            NaN  United States           US   

      state/province  population              town  distance  ...  \
0           Virgin

In [8]:
dyn_resource=boto3.resource("dynamodb")
table_name = 'landslide'
params = {
        'TableName': table_name,
        'KeySchema': [
            {'AttributeName': 'town', 'KeyType': 'HASH'},
            {'AttributeName': 'geolocation', 'KeyType': 'RANGE'}
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'town', 'AttributeType': 'S'},
            {'AttributeName': 'geolocation', 'AttributeType': 'S'}
        ],
        'ProvisionedThroughput': {
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    }
table = dyn_resource.create_table(**params)
print(f"Creating {table_name}...")
table.wait_until_exists()    

Creating landslide...


In [16]:
data=data.astype(str)
data_new=data.to_dict(orient="records")
for each_data in data_new:
    table.put_item(Item=each_data)
print("completed inserting the records")

completed inserting the records


UPDATING RECORDS IN DYNAMO DB

In [23]:
new_df=data[data['location_description']=="Unknown"]
print(new_df)

        id        date   time continent_code   country_name country_code  \
0       34  03-02-2007  Night            nan  United States           US   
179    996     3/19/09    nan             SA       Colombia           CO   
217   1186    12/28/09  Night             SA        Ecuador           EC   
252   1465  02-03-2010    nan             SA        Ecuador           EC   
371   2021     6/27/10    nan            nan      Guatemala           GT   
...    ...         ...    ...            ...            ...          ...   
1642  7487    11/18/15  01:11            nan     Costa Rica           CR   
1643  7488    10/29/15    nan            nan     Costa Rica           CR   
1653  7498     6/29/15    nan            nan         Panama           PA   
1658  7503    11/23/15    nan            nan         Panama           PA   
1661  7506  09-10-2015    nan            nan         Panama           PA   

                      state/province population  \
0                           Virginia

In [41]:
import time
t1=time.perf_counter()
num_count=0
present_status="Investigation on going"
new_df_dict=new_df.to_dict(orient="records")
for ind_data in new_df_dict:
    num_count=num_count+1
    result=update_table(ind_data,num_count,present_status)
    print(result)
    print(num_count)
t2=time.perf_counter()
print("time taken is",(t2-t1),"seconds")
    

{'num_count': Decimal('1'), 'present_status': 'Investigation on going'}
1
{'num_count': Decimal('2'), 'present_status': 'Investigation on going'}
2
{'num_count': Decimal('3'), 'present_status': 'Investigation on going'}
3
{'num_count': Decimal('4'), 'present_status': 'Investigation on going'}
4
{'num_count': Decimal('5'), 'present_status': 'Investigation on going'}
5
{'num_count': Decimal('6'), 'present_status': 'Investigation on going'}
6
{'num_count': Decimal('7'), 'present_status': 'Investigation on going'}
7
{'num_count': Decimal('8'), 'present_status': 'Investigation on going'}
8
{'num_count': Decimal('9'), 'present_status': 'Investigation on going'}
9
{'num_count': Decimal('10'), 'present_status': 'Investigation on going'}
10
{'num_count': Decimal('11'), 'present_status': 'Investigation on going'}
11
{'num_count': Decimal('12'), 'present_status': 'Investigation on going'}
12
{'num_count': Decimal('13'), 'present_status': 'Investigation on going'}
13
{'num_count': Decimal('14'), '

In [38]:
from botocore.exceptions import ClientError
def update_table(row:dict,num_count:int,present_status:str):
    try:
        response = table.update_item(
        Key={'town': row['town'], 'geolocation': row['geolocation']},
        UpdateExpression="set num_count=:r, present_status=:s",
        ExpressionAttributeValues={
            ':r': num_count, ':s': present_status},
            ReturnValues="UPDATED_NEW")
    except ClientError as err:
        print(err.response['Error']['Code'], err.response['Error']['Message'])
    else:
        return response['Attributes']

DELETING THE TABLE

In [42]:
table.delete()
print(f"Deleting {table.name}...")
table.wait_until_not_exists()

Deleting landslide...
